In [30]:
import pandas as pd
import re
pattern = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣]')
data = open('unlabeled/unlabeled_comments_1.txt', 'r', encoding='utf-8')
df = pd.read_csv('hate_train.csv', encoding='ANSI')
df['comments'] = df['comments'].str.replace('[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]','')

C:\Users\siryu\anaconda3\envs\capstone\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [31]:
df

,comments,hate
0,현재 호텔주인 심정 아 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네,1
1,한국적인 미인의 대표적인 분너무나 곱고아름다운모습그모습뒤의 슬픔을 미처 알지못했네요ㅠ,0
2,못된 넘들남의 고통을 즐겼던 넘들이젠 마땅한 처벌을 받아야지그래야 공정한 사회지심은...,1
3,화 어설펐는데 화 지나서부터는 갈수록 너무 재밌던데,0
4,사람 얼굴 손톱으로 긁은것은 인격살해이고 동영상이 몰카냐 메걸리안들 생각이 없노,1
...,...,...
14187,좌우 헬파이어 개씩 개 장착에 아파치보다 약하지만 기관포 장착임,0
14188,세금 내놓으라고 데모질 중 ㅋㅋ간첩 도둑놈 새끼들이 대통령 해처먹으니까 나도 같이...,1
14189,너가 한 말 중에,0
14190,제갈대중 ㅇㅂ,0


In [32]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [33]:
X_data = df.comments
y_data = df.hate

In [34]:
vocab_size = 10000
tokenizer = Tokenizer(num_words = vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_data)
sequences = tokenizer.texts_to_sequences(X_data)

In [35]:
print(sequences[:5])

[[843, 1, 5436, 53, 15, 1, 1, 1, 1, 95, 1], [1, 1, 2644, 1, 1, 1, 1, 1], [2645, 1, 3613, 1, 1, 1, 5437, 1, 3614, 1, 1], [1440, 1, 1440, 1, 652, 5, 5438], [36, 116, 1, 1, 1, 1, 1, 1, 401, 1709]]


In [36]:
n_of_train = int(len(sequences) * 0.8)
n_of_test = int(len(sequences) - n_of_train)

In [37]:
X_data = sequences

In [38]:
max_len = 216
data = pad_sequences(X_data, maxlen = max_len)

In [39]:
X_test = data[n_of_train:]
y_test = np.array(y_data[n_of_train:])
X_train = data[:n_of_train]
y_train = np.array(y_data[:n_of_train]) 
print("훈련용 이메일 데이터의 크기(shape): ", X_train.shape)
print("테스트용 이메일 데이터의 크기(shape): ", X_test.shape)
print("훈련용 레이블의 크기(shape): ", y_train.shape)
print("테스트용 레이블의 크기(shape): ", y_test.shape)

훈련용 이메일 데이터의 크기(shape):  (11353, 216)
테스트용 이메일 데이터의 크기(shape):  (2839, 216)
훈련용 레이블의 크기(shape):  (11353,)
테스트용 레이블의 크기(shape):  (2839,)


In [40]:
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, Embedding, Dropout, MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
model = Sequential()
model.add(Embedding(vocab_size, 32))
model.add(Dropout(0.2))
model.add(Conv1D(32, 5, strides=1, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 3)
mc = ModelCheckpoint('best_model.h5', monitor = 'val_acc', mode = 'max', verbose = 1, save_best_only = True)
history = model.fit(X_train, y_train, epochs = 10, batch_size=64, validation_split=0.2, callbacks=[es, mc])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
dropout_4 (Dropout)          (None, None, 32)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 32)          5152      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                2112      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                

In [41]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

89/89 [==============================] - 0s 2ms/step - loss: 0.9881 - acc: 0.7055

 테스트 정확도: 0.7055
